In [85]:
"""
Notebook for streaming data from a microphone in realtime

audio is captured using pyaudio
then converted from binary data to ints using struct
then displayed using matplotlib

scipy.fftpack computes the FFT

if you don't have pyaudio, then run

>>> pip install pyaudio

note: with 2048 samples per chunk, I'm getting 20FPS
when also running the spectrum, its about 15FPS
"""

import pyaudio
import os
import struct
import numpy as np
import time

# constants
CHUNK = 1024             # samples per frame
FORMAT = pyaudio.paInt16    # audio format (bytes per sample?)
CHANNELS = 1                 # single channel for microphone
RATE = 44100                 # samples per second

In [86]:
import sys
import serial

serialPort = '/dev/cu.usbmodem14301'
baudRate = 115200
usbConnection = serial.Serial(serialPort, baudRate)

def commandArduino(string):
    outputStr = string + '\n'
    outputStr = bytes(outputStr, 'utf-8')
    usbConnection.write(outputStr)
    usbConnection.reset_input_buffer()

In [92]:
# pyaudio class instance
p = pyaudio.PyAudio()

# stream object to get data from microphone
stream = p.open(
    format=FORMAT,
    channels=CHANNELS,
    rate=RATE,
    input=True,
    output=True,
    frames_per_buffer=CHUNK
)

# for measuring frame rate
frame_count = 0
frame_limit = 100
start_time = time.time()

while True:
    
    # binary data
    data = stream.read(CHUNK, exception_on_overflow = False)  
    
    # convert data to integers
    data_int = struct.unpack(str(2 * CHUNK) + 'B', data)
    
    # create np array and offset by 128
    data_np = np.array(data_int, dtype='b')[::2] + 128
    
    # volume
    delta = np.max(data_np) - np.min(data_np)
    delta_mod = int(delta ** 0.75)
    delta_bounded = min(max(delta_mod, 0), 128)

    commandArduino(f'vibrate,{delta_bounded}')
    print(delta, delta_bounded)
    
    frame_count += 1

end_time = time.time()
print(frame_limit / (end_time - start_time))

0 0
118 35
254 63
200 53
255 63
136 39
254 63
155 43
109 33
67 23
60 21
59 21
84 27
59 21
80 26
57 20
50 18
89 28
255 63
177 48
154 43
255 63
255 63
253 63
223 57
196 52
224 57
174 47
189 50
153 43
132 38
127 37
70 24
60 21
47 17
70 24
50 18
45 17
59 21
43 16
51 19
57 20
55 20
61 21
81 27
52 19
46 17
48 18
72 24
77 25
85 27
66 23
67 23
49 18
52 19
93 29
72 24
42 16
44 17
52 19
59 21
67 23
47 17
78 26
72 24
55 20
62 22
44 17
62 22
61 21
79 26
60 21
95 30
94 30
89 28
76 25
94 30
76 25
80 26
68 23
63 22
48 18
63 22
81 27
70 24
49 18
38 15
84 27
39 15
39 15
41 16
39 15
50 18
47 17
56 20
61 21
46 17
47 17
55 20
31 13
59 21
57 20
52 19
43 16
34 14
30 12
42 16
47 17
54 19
50 18
37 15
53 19
49 18
96 30
78 26
106 33
57 20
71 24
83 27
90 29
59 21
105 32
85 27
78 26
75 25
72 24
63 22
74 25
63 22
81 27
70 24
108 33
74 25
51 19
86 28
59 21
70 24
69 23
88 28
123 36
123 36
112 34
109 33
83 27
116 35
84 27
89 28
110 33
51 19
60 21
48 18
42 16
45 17
36 14
48 18
35 14
47 17
29 12
49 18
49 18
52 19
52 19

KeyboardInterrupt: 

In [90]:
test = np.fromstring(stream.read(CHUNK, exception_on_overflow = False), dtype=np.uint16)

<ipython-input-90-91d1968c9b5f>:1: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  test = np.fromstring(stream.read(CHUNK, exception_on_overflow = False), dtype=np.uint16)


In [91]:
np.histogram(test)

(array([473,   0,   0,   0,   0,   0,   0,   0,   0, 551]),
 array([    0. ,  6553.5, 13107. , 19660.5, 26214. , 32767.5, 39321. ,
        45874.5, 52428. , 58981.5, 65535. ]))